In [35]:
from gurobipy import GRB
import gurobipy as gb
import pandas as pd
import numpy as np

In [36]:
# Create the optimization model
model = gb.Model("Question 2: McKenna Logistics")

In [37]:
heavy = model.addVars(7, 9, lb=0, vtype=GRB.BINARY, name="Heavy Pallets")
medium = model.addVars(6, 9, lb=0, vtype=GRB.BINARY, name="Medium Pallets")
light = model.addVars(5, 9, lb=0, vtype=GRB.BINARY, name="Light Pallets")
truck = model.addVars(1, 9, lb=0, vtype=GRB.BINARY, name="Trucks")

In [38]:
heavy_truck_objective_function = gb.quicksum(5500*truck[0,i] for i in range(3))
medium_truck_objective_function = gb.quicksum(4700*truck[0,i] for i in range(3,6))
light_truck_objective_function = gb.quicksum(3900*truck[0,i] for i in range(6,9))
model.setObjective(heavy_truck_objective_function + medium_truck_objective_function + light_truck_objective_function, GRB.MINIMIZE)

In [39]:
model.addConstr(gb.quicksum(truck[0,i] for i in range (3)) >= 1, "Picking a Heavy Truck")
model.addConstr(gb.quicksum(truck[0,i] for i in range (3,6)) >= 1, "Picking a Medium Truck")
model.addConstr(gb.quicksum(truck[0,i] for i in range (6,9)) >= 1, "Picking a Light Truck")

<gurobi.Constr *Awaiting Model Update*>

In [40]:
for j in range (3):
    model.addConstr((4*(gb.quicksum(heavy[i,j] for i in range (7)))) + (3*(gb.quicksum(medium[i,j] for i in range (6)))) + (0.5*(gb.quicksum(light[i,j] for i in range (5)))) <= 13.5*truck[0,j], "Heavy Truck Weight Constraint")

In [41]:
for j in range (3,6):
    model.addConstr((4*(gb.quicksum(heavy[i,j] for i in range (7)))) + (3*(gb.quicksum(medium[i,j] for i in range (6)))) + (0.5*(gb.quicksum(light[i,j] for i in range (5)))) <= 12*truck[0,j], "Medium Truck Weight Constraint")

In [42]:
for j in range (6,9):
    model.addConstr((4*(gb.quicksum(heavy[i,j] for i in range (7)))) + (3*(gb.quicksum(medium[i,j] for i in range (6)))) + (0.5*(gb.quicksum(light[i,j] for i in range (5)))) <= 10*truck[0,j], "Light Truck Weight Constraint")

In [43]:
model.addConstr(gb.quicksum(heavy[i,j] for i in range(7) for j in range(9)) == 7, "Ensuring All Heavy Palets are Picked")

model.addConstr(gb.quicksum(medium[i,j] for i in range(6) for j in range(9)) == 6, "Ensuring All Medium Palets are Picked")

model.addConstr(gb.quicksum(light[i,j] for i in range(5) for j in range(9)) == 5, "Ensuring All Light Palets are Picked")

<gurobi.Constr *Awaiting Model Update*>

In [44]:
model.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: Intel(R) Core(TM) i7-8750H CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 15 rows, 171 columns and 342 nonzeros
Model fingerprint: 0xc5def884
Variable types: 0 continuous, 171 integer (171 binary)
Coefficient statistics:
  Matrix range     [5e-01, 1e+01]
  Objective range  [4e+03, 6e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+00]
Found heuristic solution: objective 42300.000000
Presolve removed 0 rows and 135 columns
Presolve time: 0.00s
Presolved: 15 rows, 36 columns, 72 nonzeros
Variable types: 0 continuous, 36 integer (9 binary)
Found heuristic solution: objective 33700.000000

Root relaxation: objective 1.917000e+04, 19 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth Int

In [45]:
# Print the decision variables
print(model.printAttr('X'))


    Variable            X 
-------------------------
Heavy Pallets[1,0]            1 
Heavy Pallets[1,6]            1 
Heavy Pallets[3,2]            1 
Heavy Pallets[3,3]            1 
Heavy Pallets[4,3]            1 
Heavy Pallets[5,0]            1 
Heavy Pallets[6,3]            1 
Medium Pallets[0,2]            1 
Medium Pallets[0,6]            1 
Medium Pallets[3,2]            1 
Medium Pallets[4,0]            1 
Medium Pallets[5,2]            1 
Medium Pallets[5,6]            1 
Light Pallets[0,0]            1 
Light Pallets[1,0]            1 
Light Pallets[2,0]            1 
Light Pallets[3,0]            1 
Light Pallets[4,0]            1 
 Trucks[0,0]            1 
 Trucks[0,2]            1 
 Trucks[0,3]            1 
 Trucks[0,6]            1 
None


In [46]:
# Value of the objective function
print("Logistic Costs: ", round(model.objVal, 2))

Logistic Costs:  19600.0
